In [12]:
 !wget https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data

--2025-05-31 16:28:49--  https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘dermatology.data’

dermatology.data        [ <=>                ]  25.36K  --.-KB/s    in 0.1s    

2025-05-31 16:28:50 (246 KB/s) - ‘dermatology.data’ saved [25964]



In [6]:
import numpy as np
import xgboost as xgb
from ucimlrepo import fetch_ucirepo 



In [10]:
  
# fetch dataset 
dermatology = fetch_ucirepo(id=33) 
  
# data (as pandas dataframes) 
X = dermatology.data.features 
y = dermatology.data.targets 
  
# metadata 
print(dermatology.metadata)
  
# variable information 
print(dermatology.variables) 

In [14]:
# label need to be 0 to num_class -1
data = np.loadtxt('./dermatology.data', delimiter=',',
        converters={33: lambda x:int(x == '?'), 34: lambda x:int(x) - 1})
sz = data.shape

train = data[:int(sz[0] * 0.7), :]
test = data[int(sz[0] * 0.7):, :]

train_X = train[:, :33]
train_Y = train[:, 34]

test_X = test[:, :33]
test_Y = test[:, 34]

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)

In [17]:
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
param["device"] = "cuda"
param["tree_method"] = "hist"
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['nthread'] = 4
param['num_class'] = 6

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist)

[0]	train-mlogloss:1.54782	test-mlogloss:1.57707
[1]	train-mlogloss:1.35705	test-mlogloss:1.39904
[2]	train-mlogloss:1.20223	test-mlogloss:1.26187
[3]	train-mlogloss:1.07060	test-mlogloss:1.14507
[4]	train-mlogloss:0.95824	test-mlogloss:1.04255


In [18]:
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist)

Test error using softmax = 0.12727272727272726
[0]	train-mlogloss:1.54782	test-mlogloss:1.57707
[1]	train-mlogloss:1.35705	test-mlogloss:1.39904
[2]	train-mlogloss:1.20223	test-mlogloss:1.26187
[3]	train-mlogloss:1.07060	test-mlogloss:1.14507
[4]	train-mlogloss:0.95824	test-mlogloss:1.04255


In [19]:
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bst.predict(xg_test).reshape(test_Y.shape[0], 6)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != test_Y) / test_Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))

Test error using softprob = 0.12727272727272726
